# Convolutional Autoencoder (Conv_AE) Model Training

In [ ]:
# Build a class for customized training process confirmation
PrintAccPerEpochs = 100

class MAE_PerEpoch(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        keras.callbacks.Callback()
        if epoch%PrintAccPerEpochs == 0:
            print("[{} Epochs] Loss(MAE) : {:.4f} ".format(epoch, logs["mae"]))

In [ ]:
# pytictoc is an useful tool for time calculation
!pip install pytictoc
from pytictoc import TicToc

In [ ]:
# Define hyperparameter
Epoch = 100

DenseUnits_1 = 300
DenseUnits_2 = 30

In [ ]:
t = TicToc()

# Design a Conv_AE model
################################################################################
input = keras.layers.Input(shape=(TrainData.shape[1], TrainData.shape[2], TrainData.shape[3]))

# Encoder
x = keras.layers.Conv2D(4 , kernel_size=(3,3), padding = 'same', activation = 'selu')(input)
x = keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(x)
x = keras.layers.Conv2D(8 , kernel_size=(3,3), padding = 'same', activation = 'selu')(x)
x = keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(x)
x = keras.layers.Conv2D(16, kernel_size=(3,3), padding = 'same', activation = 'selu')(x)
x = keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(units=DenseUnits_1, activation = 'selu')(x)

# Bottle neck layer (Latent space)
x = keras.layers.Dense(units=DenseUnits_2 , activation = 'selu')(x)

# Decoder
x = keras.layers.Dense(units=DenseUnits_1 , activation = 'selu')(x)
x = keras.layers.Dense(units=7040 , activation = 'selu')(x)
x = keras.layers.Reshape((5, 88, 16))(x)
x = keras.layers.Conv2DTranspose(16, kernel_size=(3,3), padding = 'same', activation = 'selu', strides = 2)(x)
x = keras.layers.Conv2DTranspose(8 , kernel_size=(3,3), padding = 'same', activation = 'selu', strides = 2)(x)
x = keras.layers.Conv2DTranspose(1 , kernel_size=(3,3), padding = 'same', activation = 'selu', strides = 2)(x)

Conv_AE = Model(input, x)

# Metric for optimization: Mean Absolute Error (MAE) = Reconstruction Error
Conv_AE.compile(loss="mae", optimizer='adam', metrics=['mae'])
Conv_AE.summary()

print('Latent space dimension : %d'%(DenseUnits_2))
################################################################################


# Conv_AE training
################################################################################
print("\n↓↓↓↓↓ Start Conv_AE training ↓↓↓↓↓\n")

t.tic() # training start time

tf.random.set_seed(777)
history = Conv_AE.fit(TrainData, TrainData, epochs=Epoch, verbose=0,
                      validation_data=(TestData, TestData), callbacks=[MAE_PerEpoch()])

t.toc() # training end time
time_s = t.tocvalue()
################################################################################

# Evaluation accuracy with TestData based on trained model
Loss, Final_MAE = Conv_AE.evaluate(TestData,  TestData, verbose=0)
print("\n[Final Epochs] MAE : %.4f"%(Final_MAE))
print("Training time : %.4f seconds / %.4f seconds"%(time_s, time_s/60))

In [ ]:
# Save the trained model
Conv_AE.save("/content/drive/MyDrive/your_directory/ConvAE_LsDim_%d.h5"%(DenseUnits_2))

# Save the training history
Hist = pd.DataFrame(np.zeros((Epoch,2)))
Hist.iloc[:,0] = np.array(history.history['loss'])
Hist.iloc[:,1] = np.array(history.history['val_loss'])
Hist.to_csv("/content/drive/MyDrive/your_directory/ConvAE_LsDim_%d_history.csv"%(DenseUnits_2),header=None,index=None)